In [24]:
import pandas as pd
wine_dataset=pd.read_csv("wine-dataset.csv")
wine_dataset

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,0
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,0
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,0
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,1


In [2]:
#3A Just implemented the decision tree taking data and considering the 11 attributes to be indexed between numbers 1-11 and if we want to see any output regarding thresholds and tree values we can call it using self.Tree[i] and self.Threshold[i]
#took input from the text file that is created using the given .csv file
import numpy as np
from copy import copy
class DecisionTree:
    
    def __init__(self):
        self.train=np.array([])
        self.labels=[]
        self.labelsT=[]
        self.Test=np.array([])
        self.usedThresholds={}
        for i in range(0,11):
            self.usedThresholds[i]=set()
        self.Tree=np.ones((5000,1))
        self.Thresholds=np.ones((5000,1))
        self.decisions={}
        self.Tree=-self.Tree
        self.last=0
        pass
    
    def InformationGain(self,data1,threshold,Attr,labels1):
        data=copy(data1)
        labels=copy(labels1)
        
        RLeft=np.where(data[:,Attr]>=threshold)[0]
        RRight=np.where(data[:,Attr]<threshold)[0]
        
        RH=np.where(labels==1)[0]
        RC=np.where(labels==0)[0]
        pH=float(RH.shape[0])/labels.shape[0]
        pC=float(RC.shape[0])/labels.shape[0]
        
        if pH==0 or pC==0:
            HX=0
        else:
            HX=-pH*np.log2(pH) - pC*np.log2(pC)
        
       
        
        
        LLeft=copy(labels[RLeft])
        LRight=copy(labels[RRight])
        RH=np.where(LLeft==1)[0]
        RC=np.where(LLeft==0)[0]
        
        pHL=float(RH.shape[0])/LLeft.shape[0]
        pCL=float(RC.shape[0])/LLeft.shape[0]
        if pHL==0 or pCL==0:
            HY_X_L=0
        
        else:
            HY_X_L=-pHL*np.log2(pHL) - pCL*np.log2(pCL)
            HY_X_L=HY_X_L*float(RLeft.shape[0])/data.shape[0]
        
        
     
        RH=np.where(LRight==1)[0]
        RC=np.where(LRight==0)[0]
        
        if LRight.shape[0]!=0:
            pHR=float(RH.shape[0])/LRight.shape[0]
            pCR=float(RC.shape[0])/LRight.shape[0]
        if LRight.shape[0]==0 or pHR==0 or pCR==0:
            HY_X_R=0
        else:
            HY_X_R=-pHR*np.log2(pHR) - pCR*np.log2(pCR)
            HY_X_R=HY_X_R*float(RRight.shape[0])/data.shape[0]
        
        IG=HX-HY_X_L-HY_X_R
        return IG        
        
        
       
    def findThresholdAndIG(self,data1,Attr,labels1):
        data=copy(data1)
        labels=copy(labels1)
        values=set(data[:,Attr])
        values=copy(sorted(values))
        Threshholds=[]
        for i in range(0,len(values)-1):
           Threshholds.append((values[i]+values[i+1])/2)
        if not Threshholds:
            Threshholds.append(values[0])
        Threshholds=set(Threshholds)
        Threshholds=copy(sorted(Threshholds))
        IG=[]
        
        if not Threshholds:
            return
        for threshold in Threshholds:
            IG.append(self.InformationGain(data,threshold,Attr,labels))
        
        maxIG=max(IG)
        maxThresh=IG.index(maxIG)
        
        return Threshholds[maxThresh],maxIG
            
        
                                
        
    def contructTree(self,data1,nodeNum,labels1):
            data=copy(data1)
            labels=copy(labels1)
            rows=np.where(labels==1)[0]
            rows2=np.where(labels==0)[0]
            if nodeNum>5000:
                return
            if rows.shape[0]==0 or rows2.shape[0]==0:
                self.decisions[nodeNum]=(rows.shape[0],rows2.shape[0])
                return
            
            IGA=[]
            thresholds=[]
            for attr in range(0,11):
                thresh,IG=self.findThresholdAndIG(data,attr,labels)
                IGA.append(IG)
                thresholds.append(thresh)
                
            maxIG=max(IGA)
            Attr=IGA.index(maxIG)
            thresh=thresholds[Attr]
            self.usedThresholds[Attr].add(thresh)
            self.Tree[nodeNum]=Attr
            self.Thresholds[nodeNum]=thresh
            rows=np.where(data[:,Attr]>=thresh)[0]
            rows2=np.where(data[:,Attr]<thresh)[0]
            
            dataLeft=copy(data[rows])
            dataRight=copy(data[rows2])
            if rows.shape[0]==0 or rows2.shape[0]==0:
                return
            
            labelsLeft=copy(labels[rows])
            labelsRight=copy(labels[rows2])
            self.contructTree(dataLeft,2*nodeNum,labelsLeft)
            self.contructTree(dataRight,2*nodeNum+1,labelsRight)

    
    def load_Training_data(self):

        N = 4500 
        with open("winequality.txt", "r") as file:  
            firstNlines=file.readlines()[0:N] 
            for line in firstNlines:
                line = line.strip()
                self.AllValues={}
                attrs=line.split(',')
                attr2=[float(i) for i in attrs[0:len(attrs)-1]]
                self.labels.append(int(attrs[-1]))
                attrs=copy(np.asarray(attr2))
                attrs=attrs.reshape(1,11)
                if self.train.shape[0]==0:
                    self.train=copy(attrs)
                else:
                    self.train=copy(np.vstack((self.train,attrs)))
                
        self.labels=copy(np.asarray(self.labels))
        self.labels=copy(self.labels.reshape(-1,1)) 
        self.contructTree(self.train,1,self.labels)
        #print ('the tree is ')
        #for i in range(1,12): 
        #    print (self.Tree[i],' ')
        #print( 'the thresholds are ')
        #for i in range(1,12): 
        #   print (self.Thresholds[i],' ')

ob1=DecisionTree()
ob1.load_Training_data()


In [3]:
#3B took 9 parts of given data as training data and 1 part as testing data and extracted data from .txt file assuming the 11 attributes to be 11 index numbers just for reference purpose
#took input from the text file that is created using the given .csv file
import numpy as np
from copy import copy
class DecisionTree:
    
    def __init__(self):
        self.train=np.array([])
        self.labels=[]
        self.labelsT=[]
        self.Test=np.array([])
        self.usedThresholds={}
        for i in range(0,11):
            self.usedThresholds[i]=set()
        self.Tree=np.ones((5000,1))
        self.Thresholds=np.ones((5000,1))
        self.decisions={}
        self.Tree=-self.Tree
        self.last=0
        pass
    
    def InformationGain(self,data1,threshold,Attr,labels1):
        data=copy(data1)
        labels=copy(labels1)
        
        RLeft=np.where(data[:,Attr]>=threshold)[0]
        RRight=np.where(data[:,Attr]<threshold)[0]
        
        RH=np.where(labels==1)[0]
        RC=np.where(labels==0)[0]
        pH=float(RH.shape[0])/labels.shape[0]
        pC=float(RC.shape[0])/labels.shape[0]
        
        if pH==0 or pC==0:
            HX=0
        else:
            HX=-pH*np.log2(pH) - pC*np.log2(pC)
        
       
        
        
        LLeft=copy(labels[RLeft])
        LRight=copy(labels[RRight])
        RH=np.where(LLeft==1)[0]
        RC=np.where(LLeft==0)[0]
        
        pHL=float(RH.shape[0])/LLeft.shape[0]
        pCL=float(RC.shape[0])/LLeft.shape[0]
        if pHL==0 or pCL==0:
            HY_X_L=0
        
        else:
            HY_X_L=-pHL*np.log2(pHL) - pCL*np.log2(pCL)
            HY_X_L=HY_X_L*float(RLeft.shape[0])/data.shape[0]
        
        
     
        RH=np.where(LRight==1)[0]
        RC=np.where(LRight==0)[0]
        
        if LRight.shape[0]!=0:
            pHR=float(RH.shape[0])/LRight.shape[0]
            pCR=float(RC.shape[0])/LRight.shape[0]
        if LRight.shape[0]==0 or pHR==0 or pCR==0:
            HY_X_R=0
        else:
            HY_X_R=-pHR*np.log2(pHR) - pCR*np.log2(pCR)
            HY_X_R=HY_X_R*float(RRight.shape[0])/data.shape[0]
        
        IG=HX-HY_X_L-HY_X_R
        return IG        
        
        
       
    def findThresholdAndIG(self,data1,Attr,labels1):
        data=copy(data1)
        labels=copy(labels1)
        values=set(data[:,Attr])
        values=copy(sorted(values))
        Threshholds=[]
        for i in range(0,len(values)-1):
           Threshholds.append((values[i]+values[i+1])/2)
        if not Threshholds:
            Threshholds.append(values[0])
        Threshholds=set(Threshholds)
        Threshholds=copy(sorted(Threshholds))
        IG=[]
        
        if not Threshholds:
            return
        for threshold in Threshholds:
            IG.append(self.InformationGain(data,threshold,Attr,labels))
        
        maxIG=max(IG)
        maxThresh=IG.index(maxIG)
        
        return Threshholds[maxThresh],maxIG
            
        
                                
        
    def contructTree(self,data1,nodeNum,labels1):
            data=copy(data1)
            labels=copy(labels1)
            rows=np.where(labels==1)[0]
            rows2=np.where(labels==0)[0]
            if nodeNum>5000:
                return
            if rows.shape[0]==0 or rows2.shape[0]==0:
                self.decisions[nodeNum]=(rows.shape[0],rows2.shape[0])
                return
            
            IGA=[]
            thresholds=[]
            for attr in range(0,11):
                thresh,IG=self.findThresholdAndIG(data,attr,labels)
                IGA.append(IG)
                thresholds.append(thresh)
                
            maxIG=max(IGA)
            Attr=IGA.index(maxIG)
            thresh=thresholds[Attr]
            self.usedThresholds[Attr].add(thresh)
            self.Tree[nodeNum]=Attr
            self.Thresholds[nodeNum]=thresh
            rows=np.where(data[:,Attr]>=thresh)[0]
            rows2=np.where(data[:,Attr]<thresh)[0]
            
            dataLeft=copy(data[rows])
            dataRight=copy(data[rows2])
            if rows.shape[0]==0 or rows2.shape[0]==0:
                return
            
            labelsLeft=copy(labels[rows])
            labelsRight=copy(labels[rows2])
            self.contructTree(dataLeft,2*nodeNum,labelsLeft)
            self.contructTree(dataRight,2*nodeNum+1,labelsRight)

    
    def load_Training_data(self):

        N = 4500 
        with open("winequality.txt", "r") as file:  
            firstNlines=file.readlines()[0:N] 
            for line in firstNlines:
                line = line.strip()
                self.AllValues={}
                attrs=line.split(',')
                attr2=[float(i) for i in attrs[0:len(attrs)-1]]
                self.labels.append(int(attrs[-1]))
                attrs=copy(np.asarray(attr2))
                attrs=attrs.reshape(1,11)
                if self.train.shape[0]==0:
                    self.train=copy(attrs)
                else:
                    self.train=copy(np.vstack((self.train,attrs)))
                
        self.labels=copy(np.asarray(self.labels))
        self.labels=copy(self.labels.reshape(-1,1)) 
        self.contructTree(self.train,1,self.labels)
        
    
    def load_Testing_data(self):
        
        N = 498 
        with open("winequality.txt", "r") as file:  
            lastNlines=file.readlines()[-N:] 
            for line in lastNlines:
                line=line.rstrip()   
                line=line[0:len(line)]
                attrs=line.split(',')
                attr2=[float(i) for i in attrs[0:len(attrs)-1]]
                self.labelsT.append(int(attrs[-1]))
            
                attrs=copy(np.asarray(attr2))
                attrs=attrs.reshape(1,11)
                if self.Test.shape[0]==0:
                    self.Test=copy(attrs)
                else:
                    self.Test=copy(np.vstack((self.Test,attrs)))
                
            self.labelsT=copy(np.asarray(self.labelsT))
            self.labelsT=copy(self.labelsT.reshape(-1,1))
        
        self.test(self.Test,self.labelsT)
    
    
    
    def checkAccuracy(self,value,predicted):
        value=copy(value.tolist())
        predicted=copy(predicted.tolist())
        
        correct=0
        for i in range(0,len(value)):
           
            if value[i][0]==predicted[i]:
                correct+=1
        
        return 100*(float(correct)/len(value))
        
    def findLabel(self,data1,nodeNum):
        data=copy(data1)
        value=0
       
        if self.Tree[nodeNum][0]==-1:
           
            h1,h2=self.decisions[nodeNum]
            if h1>0:
                value=1
            else:
                value=0
        elif data[int(self.Tree[int(nodeNum)][0])]>=self.Thresholds[int(nodeNum)][0] and 2*nodeNum<5000:
            value=int(self.findLabel(data,2*nodeNum))
        elif 2*nodeNum+1<5000:
            value=int(self.findLabel(data,2*nodeNum+1))
        return value
                    
    def test(self,data1,labels1):
        data=copy(data1)
        labels=copy(labels1)
        predicted=[]
        for i in range(0,data.shape[0]):
            
            value=int(self.findLabel(data[i],2))
            predicted.append(value)
            print ('testing ',i,' predicted= ',value,' actual is ',labels[i][0])
            
            
        predicted=np.asarray(predicted)
        acc=self.checkAccuracy(labels,predicted)
        print ('Accuracy is ',acc,'%')
                        
       
    #def checkValues(self):
        
ob1=DecisionTree()
ob1.load_Training_data()
ob1.load_Testing_data()


testing  0  predicted=  0  actual is  0
testing  1  predicted=  0  actual is  0
testing  2  predicted=  0  actual is  0
testing  3  predicted=  0  actual is  0
testing  4  predicted=  0  actual is  0
testing  5  predicted=  0  actual is  0
testing  6  predicted=  1  actual is  1
testing  7  predicted=  0  actual is  0
testing  8  predicted=  0  actual is  0
testing  9  predicted=  1  actual is  1
testing  10  predicted=  0  actual is  0
testing  11  predicted=  1  actual is  1
testing  12  predicted=  0  actual is  1
testing  13  predicted=  1  actual is  1
testing  14  predicted=  0  actual is  1
testing  15  predicted=  0  actual is  0
testing  16  predicted=  0  actual is  0
testing  17  predicted=  0  actual is  0
testing  18  predicted=  0  actual is  0
testing  19  predicted=  0  actual is  0
testing  20  predicted=  0  actual is  0
testing  21  predicted=  0  actual is  0
testing  22  predicted=  0  actual is  0
testing  23  predicted=  0  actual is  0
testing  24  predicted=  0